In [1]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler
import ast
import joblib
import os
from imblearn.over_sampling import RandomOverSampler

In [2]:
version = '4'
df = pd.read_csv(f'C:/Users/User/Desktop/yolofall/sum{version}.csv') 
df = df.drop(df.columns[0], axis=1)
df.head()

,"x1, y1","x2, y2","x3, y3","x4, y4","x5, y5","x6, y6","x7, y7","x8, y8","x9, y9","x10, y10","x11, y11","x12, y12","x13, y13","x14, y14","x15, y15","x16, y16","x17, y17","xb, yb","w, h",label
0,"(0.45381713, 0.56045544)","(0.0, 0.0)","(0.44965377, 0.55622727)","(0.0, 0.0)","(0.43750206, 0.55921674)","(0.4392542, 0.58792245)","(0.42670855, 0.57259524)","(0.44628447, 0.59705997)","(0.43520966, 0.5636195)","(0.4568835, 0.5807573)","(0.43991494, 0.55672157)","(0.42688385, 0.64256537)","(0.42247716, 0.6367769)","(0.44659168, 0.656644)","(0.46751222, 0.6380891)","(0.4322861, 0.70448136)","(0.4797864, 0.681184)","(0.4578125, 0.6359375)","(0.128125, 0.215625)",1
1,"(0.0, 0.0)","(0.0, 0.0)","(0.0, 0.0)","(0.50532126, 0.13661566)","(0.0, 0.0)","(0.50366473, 0.2385833)","(0.5935062, 0.23937361)","(0.47631225, 0.38038334)","(0.60480917, 0.3974566)","(0.47720233, 0.48175073)","(0.6028699, 0.52177036)","(0.51353633, 0.49829316)","(0.5753093, 0.49501076)","(0.49612182, 0.6792447)","(0.5603615, 0.67271674)","(0.5117618, 0.8419442)","(0.55959064, 0.8372415)","(0.5375, 0.48125)","(0.16875, 0.828125)",0
2,"(0.34801573, 0.651982)","(0.33817664, 0.6478184)","(0.34091985, 0.65694666)","(0.3362928, 0.63981915)","(0.34493572, 0.6633421)","(0.36429963, 0.6145185)","(0.38785157, 0.6480594)","(0.39545375, 0.6030742)","(0.43784308, 0.6587662)","(0.3765239, 0.5999789)","(0.43273005, 0.6437373)","(0.43651766, 0.56260836)","(0.4532575, 0.5787156)","(0.47318107, 0.54829246)","(0.49715033, 0.55742395)","(0.49726787, 0.5431253)","(0.51170146, 0.5357259)","(0.43046874, 0.5921875)","(0.2484375, 0.2125)",1
3,"(0.43466958, 0.5620493)","(0.44149667, 0.5522587)","(0.42657304, 0.55468667)","(0.0, 0.0)","(0.4052023, 0.5581433)","(0.4142252, 0.60092276)","(0.3907822, 0.613541)","(0.4108529, 0.67555416)","(0.4063632, 0.69424134)","(0.43879223, 0.70702225)","(0.4395338, 0.727557)","(0.41909266, 0.71533334)","(0.40236053, 0.7305919)","(0.4759139, 0.69102705)","(0.45842043, 0.72135895)","(0.5111784, 0.7421999)","(0.499053, 0.78742707)","(0.44765624, 0.66015625)","(0.1765625, 0.3203125)",1
4,"(0.0, 0.0)","(0.0, 0.0)","(0.0, 0.0)","(0.37944573, 0.22796759)","(0.0, 0.0)","(0.37634683, 0.29024535)","(0.43065563, 0.28355217)","(0.36619297, 0.36336142)","(0.0, 0.0)","(0.35861522, 0.39541686)","(0.0, 0.0)","(0.38666517, 0.44961905)","(0.42371923, 0.444026)","(0.38202173, 0.57766986)","(0.4155488, 0.5622363)","(0.39766154, 0.7029013)","(0.42670545, 0.6799125)","(0.396875, 0.459375)","(0.103125, 0.56875)",0


In [3]:
def convert_string_tuple_to_floats(s):
    if isinstance(s, str):
        return np.array(ast.literal_eval(s), dtype=np.float64)
    else:
        return s
# Apply the conversion to all relevant columns
for column in df.columns[:-1]:  # Assuming the last column is the target
    df[column] = df[column].apply(convert_string_tuple_to_floats)

# # Display the first few rows to check the conversion
# print(df.head())

# Prepare the data for the k-NN model
X = np.array([np.hstack(row) for row in df.iloc[:,:-1].values])  # Flatten the arrays into a 2D array
y = df.iloc[:,-1].values
# Apply random oversampling
ros = RandomOverSampler(random_state=42)
X, y = ros.fit_resample(X, y)
# print(X[0])
# Split the dataset into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Initialize StratifiedKFold with 5 folds
strat_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Iterate over the StratifiedKFold splits
for train_index, test_index in strat_kfold.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
# Standardize the features (mean=0, variance=1)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
print(X_train[0])

[ 3.12518790e-01  1.09026330e+00 -1.22842595e+00 -1.01836326e+00
  5.23585187e-01  1.37489584e+00 -1.11667404e+00 -9.46572654e-01
  6.22471206e-01  1.53990232e+00 -4.07629054e-01  8.79602324e-01
 -3.71864632e-01  7.99647177e-01 -1.55700822e-01  7.06280588e-01
 -5.83354791e-02  5.63612096e-01 -4.15889569e-02  5.14086958e-01
  3.94984233e-04  4.18286705e-01 -5.37500672e-01  8.34034369e-01
 -4.79931043e-01  7.94197839e-01 -3.52789128e-01  4.92509513e-01
 -1.45609307e-01  3.74937951e-01 -3.56384943e-01  3.85531555e-01
 -9.95145288e-03  2.68397916e-01 -2.59652022e-01  7.48246195e-01
 -8.43177813e-01 -1.08132405e+00]


In [4]:
# Create an SVM classifier
svm = SVC(kernel='rbf')  # non-linear kernel SVM

# Train the SVM classifier
svm.fit(X_train, y_train)

# Make predictions on the test data
y_pred = svm.predict(X_test)

# Evaluate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Display the predictions and the actual labels
print("\nPredictions:", y_pred)
print("Actual labels:", y_test)

Accuracy: 0.98

Predictions: [0 1 1 ... 0 0 0]
Actual labels: [1 1 1 ... 0 0 0]


In [5]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# Assuming y_test and y_pred are already defined
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Confusion Matrix:")
print(cm)
print("\nAccuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Confusion Matrix:
[[2256   43]
 [  29 2269]]

Accuracy: 0.9843376114857516
Precision: 0.981401384083045
Recall: 0.9873803307223673
F1 Score: 0.9843817787418655


In [6]:
# Export the trained KNN model to a file
save_path_svm = f'C:/Users/User/Desktop/yolofall/new_svm_model_{version}.pkl'
save_path_scaler = f'C:/Users/User/Desktop/yolofall/new_svm_scaler_{version}.pkl'
joblib.dump(svm, save_path_svm)
joblib.dump(scaler, save_path_scaler)

['C:/Users/User/Desktop/yolofall/new_svm_scaler_4.pkl']